# Exploratory Data Analysis

*Coming Soon...*

## Introduction

*Coming Soon...*

## Mel Spectrogram

A spectrogram is a 2D representation of a signal, having time on the x-axis and frequency on the y-axis.

*Coming Soon...*

## Other Features

Time Domain Features
 * Central Moments: (mean, standard deviation, skewness and kurtosis of the amplitude of the signal)
 * Zero Crossing Rate
 * Root Mean Square Energy
 * Tempo
 
Frequency Domain Features
 * Mel-Frequency Cepstral Coefficients
 * Chroma Features
 * Spectral Centroid
 * Spectral Band-with

*Coming Soon...*